In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, plot_confusion_matrix,fbeta_score, make_scorer
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

import time

In [5]:
def print_distribution(arr, print_nl=True):
    arr = np.unique(arr, return_counts=True)[1] / len(arr)
    for i in range(arr.shape[0]):
        print("Classe %d: %.2f%%" %(i, arr[i]*100))

    if print_nl:
        print("\n")

In [6]:
time_begin = time.time()
data = pd.read_csv("wine_quality.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed.acidity         6497 non-null   float64
 1   volatile.acidity      6497 non-null   float64
 2   citric.acid           6497 non-null   float64
 3   residual.sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free.sulfur.dioxide   6497 non-null   float64
 6   total.sulfur.dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 609.2 KB


## Divisão de base de dados e separação de base

In [12]:
#X = data[["fixed.acidity","volatile.acidity","citric.acid","residual.sugar","chlorides","free.sulfur.dioxide","total.sulfur.dioxide","density","pH","sulphates","alcohol"]]
#y = data[["quality"]]
X = data.iloc[:, 0:11].values
y = data.iloc[:, 11].values

In [23]:
print("Proporções por classe no dataset em geral:")
print_distribution(y)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=199)
fold = 0
for train, test in kf.split(X, y):
    print("Fold %d" %(fold))
    print_distribution(quality[train], print_nl=(fold != 2))
    fold += 1

Proporções por classe no dataset em geral:
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.97%
Classe 6: 0.08%


Fold 0
Classe 0: 0.46%
Classe 1: 3.34%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.96%
Classe 6: 0.09%


Fold 1
Classe 0: 0.46%
Classe 1: 3.34%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.96%
Classe 6: 0.09%


Fold 2
Classe 0: 0.46%
Classe 1: 3.34%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.61%
Classe 5: 2.98%
Classe 6: 0.07%
Fold 3
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.65%
Classe 4: 16.62%
Classe 5: 2.98%
Classe 6: 0.07%


Fold 4
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.66%
Classe 4: 16.61%
Classe 5: 2.98%
Classe 6: 0.07%


Fold 5
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.66%
Classe 4: 16.61%
Classe 5: 2.98%
Classe 6: 0.07%


Fold 6
Classe 0: 0.46%
Classe 1: 3.32%
Classe 2: 32.91%
Classe 3: 43.66%
Classe 4: 16.61%
Classe 5

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


In [16]:
kf_results = list()

from sklearn.metrics import make_scorer, f1_score,precision_score,recall_score
scores = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'macro'),
           'recall': make_scorer(recall_score, average = 'macro'),
           'f1_macro': make_scorer(f1_score, average = 'macro'),
           'f1_weighted': make_scorer(f1_score, average = 'weighted')}

In [19]:
for train_ix, test_ix in kf.split(X,y):
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    #cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    model = DecisionTreeClassifier(random_state=1)
    # define search space
    parameters = dict()
    parameters["max_leaf_nodes"] = list(range(2, 50))
    parameters["min_samples_split"] = [2, 4]
    search = GridSearchCV(model, parameters, scoring="accuracy", cv=5, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    kf_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


>acc=0.534, est=0.509, cfg={'max_leaf_nodes': 4, 'min_samples_split': 2}
>acc=0.538, est=0.508, cfg={'max_leaf_nodes': 4, 'min_samples_split': 2}


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


>acc=0.509, est=0.505, cfg={'max_leaf_nodes': 7, 'min_samples_split': 2}


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


...

>acc=0.520, est=0.514, cfg={'max_leaf_nodes': 3, 'min_samples_split': 2}


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


>acc=0.512, est=0.510, cfg={'max_leaf_nodes': 3, 'min_samples_split': 2}


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


>acc=0.531, est=0.511, cfg={'max_leaf_nodes': 3, 'min_samples_split': 2}
>acc=0.505, est=0.505, cfg={'max_leaf_nodes': 3, 'min_samples_split': 2}
>acc=0.512, est=0.514, cfg={'max_leaf_nodes': 3, 'min_samples_split': 2}
>acc=0.508, est=0.512, cfg={'max_leaf_nodes': 3, 'min_samples_split': 2}


In [21]:
print('Accuracy: %.3f (%.3f)' % (np.mean(kf_results), np.std(kf_results)))

Accuracy: 0.519 (0.011)


In [22]:
time_end = time.time()
print((time_end - time_begin)/60)

24.052873226006827
